(ch2:finetune)=
# 2.5 Fine tune, discover, and visualize

In [ ]:
mode = "svg"  # output format for figs

import matplotlib

font = {'family' : 'Dejavu Sans',
        'weight' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
import os
import urllib
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from graspologic.utils import import_edgelist
import numpy as np
import glob
from tqdm import tqdm

# the AWS bucket the data is stored in
BUCKET_ROOT = "open-neurodata"
parcellation = "Schaefer400"
FMRI_PREFIX = "m2g/Functional/BNU1-11-12-20-m2g-func/Connectomes/" + parcellation + "_space-MNI152NLin6_res-2x2x2.nii.gz/"
FMRI_PATH = os.path.join("datasets", "fmri")  # the output folder
DS_KEY = "abs_edgelist"  # correlation matrices for the networks to exclude

def fetch_fmri_data(bucket=BUCKET_ROOT, fmri_prefix=FMRI_PREFIX,
                    output=FMRI_PATH, name=DS_KEY):
    """
    A function to fetch fMRI connectomes from AWS S3.
    """
    # check that output directory exists
    if not os.path.isdir(FMRI_PATH):
        os.makedirs(FMRI_PATH)
    # start boto3 session anonymously
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    # obtain the filenames
    bucket_conts = s3.list_objects(Bucket=bucket, 
                    Prefix=fmri_prefix)["Contents"]
    for s3_key in tqdm(bucket_conts):
        # get the filename
        s3_object = s3_key['Key']
        # verify that we are grabbing the right file
        if name not in s3_object:
            op_fname = os.path.join(FMRI_PATH, str(s3_object.split('/')[-1]))
            if not os.path.exists(op_fname):
                s3.download_file(bucket, s3_object, op_fname)

def read_fmri_data(path=FMRI_PATH):
    """
    A function which loads the connectomes as adjacency matrices.
    """
    fnames = glob.glob(os.path.join(path, "*.csv"))
    fnames.sort()
    # import edgelists with graspologic
    # edgelists will be all of the files that end in a csv
    networks = [import_edgelist(fname) for fname in tqdm(fnames)]
    return np.stack(networks, axis=0)

In [ ]:
fetch_fmri_data()
As = read_fmri_data()

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from graspologic.utils import pass_to_ranks

def remove_isolates(A):
    """
    A function which removes isolated nodes from the 
    adjacency matrix A.
    """
    degree = A.sum(axis=0)  # sum along the rows to obtain the node degree
    out_degree = A.sum(axis=1)
    A_purged = A[~(degree == 0),:]
    A_purged = A_purged[:,~(degree == 0)]
    return A_purged

class CleanData(BaseEstimator, TransformerMixin):

    def fit(self, X):
        return self

    def transform(self, X):
        Acleaned = remove_isolates(X)
        A_abs_cl = np.abs(Acleaned)
        self.A_ = A_abs_cl
        return self.A_

class FeatureScaler(BaseEstimator, TransformerMixin):
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        A_scaled = pass_to_ranks(X)
        return (A_scaled)

num_pipeline = Pipeline([
    ('cleaner', CleanData()),
    ('scaler', FeatureScaler()),
])

As_xfm = [num_pipeline.fit_transform(A) for A in As]

In [ ]:
from graspologic.embed import MultipleASE
from graspologic.plot import pairplot
from matplotlib import pyplot as plt

embedding = MultipleASE(n_components=5, svd_seed=0).fit_transform(As_xfm)
fig = pairplot(embedding, title="(A) Multiple spectral embedding of all connectomes")

fig.tight_layout()

In [ ]:
from graspologic.cluster import AutoGMMCluster

labels = AutoGMMCluster(max_components=10, random_state=0).fit_predict(embedding)
fig = pairplot(embedding, labels=labels,
                title="Multiple spectral embedding of all connectomes", 
                legend_name="Predicted Clusters")

fig.tight_layout()

fname = "mase"
if mode != "png":
    os.makedirs(f"Figures/{mode:s}", exist_ok=True)
    fig.savefig(f"Figures/{mode:s}/{fname:s}.{mode:s}")

os.makedirs("Figures/png", exist_ok=True)
fig.savefig(f"Figures/png/{fname:s}.png")

In [ ]:
from urllib import request
import json
import pandas as pd
from pathlib import Path

coord_dest = os.path.join(FMRI_PATH, "coordinates.json")
request.urlretrieve("https://github.com/neurodata/neuroparc/" + "raw/master/atlases/label/Human/Metadata-json/" + parcellation + "_space-MNI152NLin6_res-2x2x2.json", coord_dest)

with open(coord_dest) as coord_f:
    coords = []
    for roiname, contents in json.load(coord_f)["rois"].items():
        try:
            if roiname != "0":
                coord_roi = {"x" : contents["center"][0], "y" : contents["center"][1], "z" : contents["center"][2]}
                coords.append(coord_roi)
        except:
            continue
Path("./Images/").mkdir(parents=True, exist_ok=True)
request.urlretrieve("https://github.com/neurodata/graph-stats-book/" +
                    "blob/master/network_machine_learning_in_python/" +
                    "foundations/ch2/Images/lobes.png?raw=true", "./Images/lobes.png")
            
coords_df = pd.DataFrame(coords)

In [ ]:
import matplotlib.image as mpimg
import seaborn as sns

coords_df["Community"] = labels
coords_df['Community'] = coords_df['Community'].astype('category')
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
axs[0].imshow(mpimg.imread('./Images/lobes.png'))
axs[0].set_axis_off()
sns.scatterplot(x="y", y="z", data=coords_df, hue="Community", ax=axs[1])
axs[1].set_ylabel("Bottom to Top (mm)")
axs[1].set_xlabel("Back to Front (mm)")
axs[1].legend(title="Predicted Community", bbox_to_anchor=(1, 1))

fig.tight_layout()

fname = "brain_preds"
if mode != "png":
    fig.savefig(f"Figures/{mode:s}/{fname:s}.{mode:s}")

fig.savefig(f"Figures/png/{fname:s}.png")

In [ ]:
import contextlib
import datasets.dice as dice
from sklearn.metrics import confusion_matrix
from graphbook_code import cmaps

group_dest = os.path.join("./datasets/", "Yeo-7_space-MNI152NLin6_res-2x2x2.nii.gz")
request.urlretrieve("https://github.com/neurodata/neuroparc/" + "blob/master/atlases/label/Human/" +
                    "Yeo-7_space-MNI152NLin6_res-2x2x2.nii.gz?raw=true", group_dest)
roi_dest = os.path.join("./datasets/", parcellation + "_space-MNI152NLin6_res-2x2x2.nii.gz")
request.urlretrieve("https://github.com/neurodata/neuroparc/" + "blob/master/atlases/label/Human/" + 
                    parcellation + "_space-MNI152NLin6_res-2x2x2.nii.gz?raw=true", roi_dest)

dicemap, _, _ = dice.dice_roi("./datasets/", "./datasets", 
                              "Yeo-7_space-MNI152NLin6_res-2x2x2.nii.gz", 
                              parcellation + "_space-MNI152NLin6_res-2x2x2.nii.gz",
                              verbose=False)
actual_cluster = np.argmax(dicemap, axis=0)[1:] - 1

# make confusion matrix
cf_matrix = confusion_matrix(actual_cluster, labels)

In [ ]:
# and plot it
fig, ax = plt.subplots(1,1, figsize=(5, 4))
sns.heatmap(cf_matrix, cmap=cmaps["sequential"], ax=ax)
ax.set_title("Confusion matrix")
ax.set_ylabel("True Parcel")
ax.set_xlabel("Predicted Community")
ax.set_xlim((0, 4))
fig.tight_layout()

fname = "cf_mtx"
if mode != "png":
    fig.savefig(f"Figures/{mode:s}/{fname:s}.{mode:s}")

fig.savefig(f"Figures/png/{fname:s}.png")